In [84]:
import json
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
from nltk.corpus import stopwords

In [9]:
tweets = sc.textFile("/Users/mayankkedia/code/epl-twitter/sample_tweets/smaller_sample/")
tweets = tweets.map(lambda x: x.rstrip(",")).map(lambda x: json.loads(x))
tweets.cache()

### Finding the top words used in tweets (English)

In [ ]:

tweet_texts = tweets.filter(lambda x: x['lang']=='en').map(lambda x: x['text'])

In [8]:
from nltk.corpus import stopwords

### Splitting tweets into words

In [88]:
new_stopwords = ["rt", "", "//t", "1", "it's", 'u', 'go', 'like', 'come', 'next','&amp', '#bpl', 
                "i'm" , "he's", "get", "still", "back", 'https', 'vs', "can't", "old"]
new_stopwords.extend(stopwords.words('english'))

In [82]:
len(new_stopwords)

170

In [89]:
def parseAndRemoveStopWords(text):
    """
    A function to parse and remove stop words. 
    @TODO: needs to remove urls, numbers
    """
    t = text.encode('ascii','ignore')
    t = t.replace(";"," ").replace(":"," ").replace('"',' ').replace('-',' ')
    t = t.replace(',',' ').replace('.',' ').replace("  ", " ")
    t = t.lower().split(" ")
    ## remove urls
    ## remove numbers
    ## Remove @, #, double blanks
    stop = set(new_stopwords)
    return [i for i in t if i not in stop]

In [90]:
tweet_words = tweet_texts.flatMap(lambda x: parseAndRemoveStopWords(x))

### Doing word counts

In [100]:
word_counts = tweet_words.map(lambda x: (x, 1)).reduceByKey(lambda x,y :x+y)

In [101]:
top_words = word_counts.takeOrdered(100, lambda x:-x[1])

In [102]:
top_words = set([x[0] for x in top_words])

In [103]:
print top_words

set(['#lcfc', 'leicester', '#munlei', "mahrez's", 'manchester', 'follow', 'yes', 'toast', 'lb', 'title', 'rojo', "don't", 'fans', 'winning', 'going', '4', 'good', 'watching', '#leicester', 'game', 'fan', 'half', "71'", 'mahrez', 'gray', "we'll", "@juanmata8's", 'team', '@caspar_lee', 'old', '#leicestercity', 'play', 'claudio', 'see', 'even', '@premierleague', '@manutd', 'away', 'beaten', 'juan!', '3', 'score', 'goes', 'co/sw2zov1qjq', 'rooney', 'never', 'co/ocb6we5b9y', 'substitution', 'change', 'ranieri', 'great', 'minutes', 'makes', 'comes', '@fellaini', 'love', '@anderherrera', 'leicester!', 'win', 'powerful', 'one', 'albrighton', 'co/nb0c06lqg6', '@lcfc', 'city', 'would', 'top', 'live', 'today', '@simpiyrooney', 'league', 'season', "75'", 'alive', 'effort', 'schlupp', 'congratulations', 'utd', '@d_degea', 'appearance', 'morgan', 'trafford', '#mufc', 'stream', 'premier', 'mind', '100th', 'someone', 'want', 'need', 'united', 'goal', 'make', 'gets', '#mnulei', 'fellaini', 'football', 

## Using the top words and finding their frequencies by minute

In [75]:
# The timestamp in milliseconds is converted to timestamp in seconds
text_time = tweets.map(lambda x: (x['text'], int(x['timestamp_ms'])//1000))

In [97]:
list_words_time = text_time.map(lambda x: (parseAndRemoveStopWords(x[0]), x[1]))
print list_words_time.take(1)

[(['@atanasi_', 'gooooooooooooooal!\n\nleicester'], 1462112992)]


In [105]:
x = [0, 1]
y = x
print x,y
def changeX(a):
    a.append(4)
def changeY(b):
    b = [3,4]
changeX(x)
changeY(y)
print x,y

[0, 1] [0, 1]
[0, 1, 4] [0, 1, 4]


In [109]:
def keep_only_important_words(tup):
    word_list = tup[0]
    imp_word_list = [w for w in word_list if w in top_words]
    return (imp_word_list, tup[1])

In [130]:
imp_words_time = list_words_time.map(keep_only_important_words)

In [119]:
def change_timestamp_to_time(tup):
    """
    Convert timestamp to dayhourminute
    """
    dt = datetime.utcfromtimestamp(tup[1])
    
    return (str(dt.day)+str(dt.hour)+str(dt.minute), tup[0])

In [131]:
imp_words_time = imp_words_time.map(change_timestamp_to_time)

In [132]:
imp_words_time.take(10)

[('11429', []),
 ('11429', ['leicester']),
 ('11429', ['leicester']),
 ('11429',
  ['@manutd',
   "@juanmata8's",
   '100th',
   'appearance',
   '#mufc',
   'congratulations',
   'juan!',
   'co/sw2zov1qjq']),
 ('11429',
  ['leicester',
   'man',
   'utd',
   'title',
   'leicester',
   'city',
   'claudio',
   'ranieri']),
 ('11429', ['leicester']),
 ('11429', ['leicester', 'fans', 'manchester', 'united', 'league']),
 ('11429', ['leicester', 'old']),
 ('11429', ['@manutd', 'substitution', 'leicester', 'gray', '#mufc']),
 ('11429', ['leicester'])]

In [144]:
def concatenateLists(x,y):
    x.extend(y)
    return x

In [145]:
words_by_time = imp_words_time.reduceByKey(concatenateLists)

In [147]:
words_by_time.take(1)



[('11438',
  ['united',
   'leicester',
   'city',
   'premier',
   'leicester',
   '@manutd',
   '#leicestercity',
   'leicester',
   '@lcfc',
   'premier',
   'leicester',
   'leicester',
   'manchester',
   'united',
   'leicester',
   'city',
   'manchester',
   'united',
   'leicester',
   'city',
   'leicester',
   '#mufc',
   'leicester',
   'old',
   'trafford',
   'leicester',
   'half',
   'goal',
   'morgan',
   'title',
   '#lcfc',
   'leicester',
   'leicester',
   'leicester',
   'win',
   'season',
   '#lcfc',
   'leicester',
   'leicester',
   'leicester',
   '#lcfc',
   'win',
   'today',
   "we'll",
   'one',
   'follow',
   'man',
   'fans',
   'leicester',
   'fans',
   '#mufc',
   'leicester',
   'leicester',
   '@manutd',
   'leicester',
   'man',
   'united',
   'leicester',
   'city',
   'live',
   'stream',
   '#mnulei',
   '#lcfc',
   '#leicester',
   'love',
   'leicester',
   'city',
   'fans',
   '#lcfc',
   'old',
   'manchester',
   '@lcfc',
   '@lcfc',
 

In [168]:
from collections import Counter
def word_counts(words):
    return dict(Counter(words))

In [169]:
word_freq_by_time = words_by_time.map(lambda x: (x[0], dict(Counter(x[1]))))

In [170]:
word_freq_by_time.take(1)

[('11438',
  {'#lcfc': 187,
   '#leicester': 67,
   '#leicestercity': 35,
   '#mnulei': 88,
   '#mufc': 322,
   '#munlei': 36,
   '100th': 17,
   '3': 20,
   '4': 54,
   "71'": 14,
   "75'": 29,
   '@anderherrera': 38,
   '@caspar_lee': 13,
   '@d_degea': 12,
   '@fellaini': 36,
   "@juanmata8's": 14,
   '@lcfc': 143,
   '@manutd': 212,
   '@premierleague': 14,
   '@simpiyrooney': 11,
   'albrighton': 162,
   'alive': 20,
   'appearance': 17,
   'away': 17,
   'beaten': 10,
   'change': 40,
   'city': 164,
   'claudio': 17,
   'co/nb0c06lqg6': 13,
   'co/ocb6we5b9y': 11,
   'co/sw2zov1qjq': 14,
   'comes': 99,
   'congratulations': 16,
   "don't": 30,
   'effort': 14,
   'even': 22,
   'fan': 43,
   'fans': 65,
   'fellaini': 26,
   'follow': 17,
   'football': 17,
   'game': 27,
   'gets': 34,
   'goal': 36,
   'goes': 15,
   'going': 15,
   'good': 14,
   'gray': 9,
   'great': 13,
   'half': 20,
   'juan!': 16,
   'lb': 11,
   'league': 140,
   'leicester': 1300,
   'leicester!': 25

### To do
- shorten words to their roots
- count mahrez, mahrez's as the same
- trailing 's'
- run this on large dataset
- choose top 50 words to work with